In [3]:
import os
from dotenv import load_dotenv

load_dotenv()


os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_API_KEY"] = os.getenv("LANGCHAIN_API_KEY")

Python-dotenv could not parse statement starting at line 1
Python-dotenv could not parse statement starting at line 5


In [7]:
from langchain_community.document_loaders import PyPDFLoader

loader = PyPDFLoader('LLM Scenario.pdf')
docs = loader.load()
docs = docs[1:]

In [10]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size = 1000, chunk_overlap = 200)
documents = text_splitter.split_documents(docs)
documents

[Document(page_content='Loans Description  \nSmart bank mainl y has 5 type s of loan schemes.  \n1. Overdra ft Facilities  \n2. Housing Loans  \n3. Education Loans  \n4. Pensioners ’ Loans  \n5. Personal Loans  \n1. Over draft  Facilities  \nAn overdraft is the ideal way to manage your cash flow at competitive rates of interest at speed. It is \nlinked to your current account and you can use as much as you need, up to your limit. You can also apply \nfor and either increase or decrease your limit through Internet banking. The controls are in your hands.  \n• An overdraft is quick and easy to arrange  \n• The cash is available when you need it  \n• You only pay interest on what you use, not on the full amount of your set limit  \n• Sole proprietors can apply for an overdraft or increase their limits   \n2. Housing Loans  \n2.1. Housing Loan Sche me \n• Purchase a land to construct a house later  \n• Purchase a land and to construct a house  \n• Construct a house in a land owned by the b

In [12]:
from langchain_community.embeddings import OpenAIEmbeddings
from langchain_community.vectorstores import FAISS

db = FAISS.from_documents(documents, OpenAIEmbeddings())

c:\Users\Janith\Desktop\intellihack\intellihackvenv\lib\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAIEmbeddings`.
  warn_deprecated(


In [13]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model= "gpt-3.5-turbo")

In [19]:
from langchain_core.prompts import ChatPromptTemplate

user_prompt =  """ 
    Answer the following question based only on the provided context and general banking knowlage.
    Think step by step before providing a detailed answer.
    <context> {context} </context>
    Question: {input}
    """

prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful bank assistant. Please response to the user queries."),
    ("user",user_prompt)
])

In [20]:
from langchain.chains.combine_documents import create_stuff_documents_chain

document_chain = create_stuff_documents_chain(llm, prompt)

In [21]:
retriever = db.as_retriever()

In [25]:
from langchain_core.prompts import MessagesPlaceholder
from langchain.chains import create_history_aware_retriever

contextualize_q_system_prompt = """Given a chat history and the latest user question \
which might reference context in the chat history, formulate a standalone question \
which can be understood without the chat history. Do NOT answer the question, \
just reformulate it if needed and otherwise return it as is."""
contextualize_q_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", contextualize_q_system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)
history_aware_retriever = create_history_aware_retriever(
    llm, retriever, contextualize_q_prompt
)

In [29]:
from langchain.chains import create_retrieval_chain

qa_system_prompt = """You are a bank assistant for question-answering tasks. \
Use the following pieces of retrieved context and general banking knowlage to answer the question. \
Give output in tone of friendly bank assistant.

{context}"""
qa_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", qa_system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)
question_answer_chain = create_stuff_documents_chain(llm, qa_prompt)

rag_chain = create_retrieval_chain(history_aware_retriever, question_answer_chain)


In [30]:
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

store = {}


def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]


conversational_rag_chain = RunnableWithMessageHistory(
    rag_chain,
    get_session_history,
    input_messages_key="input",
    history_messages_key="chat_history",
    output_messages_key="answer",
)

In [32]:
conversational_rag_chain.invoke(
    {"input": "What are the terms and conditions for that loan?"},
    config={"configurable": {"session_id": "abc123"}},
)["answer"]

Parent run 8453b99e-922a-4fe3-8cae-9c270572ad03 not found for run c95d49ea-33ec-4cc3-9228-1b6043b48c1c. Treating as a root run.


"The terms and conditions for the Housing Loan Scheme for Permanent Cadre Employees are as follows:\n\n1. Repayment Period:\n   - Maximum of 25 years. The loan should be fully settled before the borrower's retirement day.\n\n2. Security/Collateral:\n   - Primary mortgage over the property to be developed.\n   - For a condominium apartment, the property will be mortgaged to the bank after completion of construction. A tripartite agreement should be signed between the developer, borrower, and the bank to continue the process until the mortgage of the property has been fully recovered.\n\nPlease note that Smart Bank reserves the authority to change, add, or revise any condition, term, or statement in this facility. For more detailed information or clarification on the terms and conditions, you can contact the Manager at your nearest Smart Branch or call our customer service center at 1234.\n\nIf you have any specific questions or need further assistance, feel free to ask. We are here to h